In [1]:
! pip install bert-tensorflow
! pip install tensorflow_text

     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 4.9 MB 5.2 MB/s 


In [2]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
import keras
from bert.tokenization import FullTokenizer
import re
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# Some constants
index_to_tag = {0: '-PAD-', 1: 'ADJ', 2: 'ADP', 3: 'ADV', 4: 'CONJ', 5: 'DET', 6: 'NOUN', 7: 'NUM', 8: 'PRON', 9: 'PRT', 10: 'PUNCT', 11: 'VERB', 12: 'X'}
max_seq_length = 72
checkpoint_path = 'drive/MyDrive/my_best_model_8.hdf5'
n_tags = 13

In [4]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

In [5]:
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)

In [6]:
bert_inputs = [input_word_ids, input_mask, segment_ids]
bert_output = bert_model(bert_inputs)[1]
outputs = keras.layers.Dense(n_tags, activation=keras.activations.softmax)(bert_output)
model = keras.models.Model(inputs=bert_inputs, outputs=outputs)
model.summary(200)

Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                     Output Shape                                Param #                 Connected to                                                      
 input_word_ids (InputLayer)                                      [(None, 72)]                                0                       []                                                                
                                                                                                                                                                                                        
 input_mask (InputLayer)                                          [(None, 72)]                                0                       []                                             

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
model.load_weights(checkpoint_path)

# Testing model with validation data

In [9]:
! git clone https://github.com/Vishisht-rao/POS-Tagger-Using-Transformers.git

Cloning into 'POS-Tagger-Using-Transformers'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 61 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [10]:
val_input_ids = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_input_ids.npz')
val_input_masks = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_input_masks.npz')
val_type_ids = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_type_ids.npz')
val_tags = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_tags.npz')

In [11]:
val_input_ids = val_input_ids['arr_0']
val_input_masks = val_input_masks['arr_0']
val_type_ids = val_type_ids['arr_0']
val_tags = val_tags['arr_0']

In [12]:
val_tags_pred = model.predict([val_input_ids, val_input_masks, val_type_ids], batch_size=16)

In [21]:
val_tags_pred_argmax = val_tags_pred.argmax(-1)
val_tags_argmax = val_tags.argmax(-1)

In [22]:
val_tags_pred_argmax[0]

array([ 0, 11,  6,  6,  3, 11,  5,  6,  6,  2,  6,  7,  2,  7, 10,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0])

In [23]:
val_tags_argmax[0]

array([ 0, 11,  6,  6,  3, 11,  5,  6,  6,  2,  6,  7,  2,  7, 10,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0])

In [24]:
val_tags_argmax = val_tags_argmax.flatten()
val_tags_pred_argmax = val_tags_pred_argmax.flatten()

In [25]:
val_tags_argmax_list = []
val_tags_pred_argmax_list = []

for i in range(val_tags_argmax.shape[0]):
  if val_tags_argmax[i] != 0:
    val_tags_argmax_list.append(val_tags_argmax[i])
    val_tags_pred_argmax_list.append(val_tags_pred_argmax[i])

val_tags_argmax_list = np.array(val_tags_argmax_list)
val_tags_pred_argmax_list = np.array(val_tags_pred_argmax_list)

In [28]:
print(classification_report(val_tags_argmax_list, val_tags_pred_argmax_list, zero_division=1))

              precision    recall  f1-score   support

           0       0.00      1.00      0.00         0
           1       0.94      0.93      0.94     10899
           2       0.97      0.97      0.97     18427
           3       0.95      0.94      0.95      6948
           4       0.98      0.97      0.97      4781
           5       0.98      0.98      0.98     17130
           6       0.96      0.97      0.97     38589
           7       0.96      0.96      0.96      2813
           8       0.97      0.97      0.97      6043
           9       0.94      0.93      0.93      4176
          10       0.98      0.97      0.97     19251
          11       0.98      0.97      0.97     23739
          12       0.92      0.89      0.91       752

    accuracy                           0.96    153548
   macro avg       0.89      0.96      0.88    153548
weighted avg       0.97      0.96      0.97    153548



In [29]:
index_to_tag

{0: '-PAD-',
 1: 'ADJ',
 2: 'ADP',
 3: 'ADV',
 4: 'CONJ',
 5: 'DET',
 6: 'NOUN',
 7: 'NUM',
 8: 'PRON',
 9: 'PRT',
 10: 'PUNCT',
 11: 'VERB',
 12: 'X'}